# Ładownie danych

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing
import seaborn as sns # visualization library

data = pd.read_csv("input/paysim.csv")

Podgląd kluku pierwszych wierszy oraz struktury danych

In [ ]:
data.head()

In [ ]:
data.describe()

Informacje o typie danych w poszczególnych kolumnach

In [ ]:
data.info()

In [ ]:
data['isFlaggedFraud'].sum()

W danych dostępna jest kolumna isFlaggedFraud, która jest oznaczana gdy przelew był na kwoty większe niż 200 000. Flaga ta została pominięta, ponieważ występuje ona tylko 16 razy w całym zbiorze danych. Pomijane są również kolumny nameOrig oraz nameDest.

In [ ]:
data.drop(['nameOrig','nameDest','isFlaggedFraud'], axis = 1, inplace=True)

Wizualizacja danych

In [ ]:
sns.countplot(data['type'], hue = data['isFraud'])

Można tutaj zauważyć, że oszustwa są tylko w typach TRANSFER oraz CASH_OUT.

In [ ]:
data[data['isFraud']==1].groupby('type').count()

# Przygotowanie danych

Przed przystąpieniem do algorytmu powinniśmy znormalizować dane.

In [ ]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_toScale = data[['amount', 'oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest']]
new_X = sc.fit(X_toScale)
X_scaled = new_X.transform(X_toScale)

Tworzenie DataFrame ze znormalizowanych danych.

In [ ]:
scaled_df = pd.DataFrame(X_scaled, columns=['amount', 'oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest'])

Pozostałą kolumnę 'type' przekształcamy na typ dummy oraz łączymy z finalnym DataFrame.

In [ ]:
dummy_df = pd.DataFrame(pd.get_dummies(data['type']))
final_df = scaled_df.join(dummy_df, how = 'outer')

Ostatecznie dane prezentują się następująco.

In [ ]:
final_df.head(5)

# Algorytm

Po przygotowaniu danych możemy przystąpić do stworzenia modelu. Wkorzystamy model RandomForestClassifier ze 100 drzewami.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(n_estimators=100)

Następnie dzielimy dane na próbę uczącą i trenującą.

In [ ]:
from sklearn.model_selection import train_test_split

X = final_df
y = data['isFraud']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

Ostatecznie przechodzimy do uczenia naszego modelu, przez podanie mu danych.

In [ ]:
model = rfc.fit(X_train,y_train)
predictions = model.predict(X_test)

# Wyniki

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

print(classification_report(y_test,predictions))
print(confusion_matrix(y_test, predictions))